<a href="https://colab.research.google.com/github/ankit0802/hello_world/blob/master/DOG_BREED_CLASSIFIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading key-value pair dictionaries of labels and file paths can be done as follow

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from sklearn.datasets import load_files 
from keras.utils import np_utils
import numpy as np
from glob import glob

# defining the funtion to load train  ,tesy, and validate data set #

def load_dataset(path):
  data = load_files(path)
  dog_files = np.array(data['filenames'])
  dog_targets = np_utils.to_categorical(np.array(data['target']),133)
  return dog_files ,dog_targets
#### load train ,test ,and validation datasets ####
train_files, train_targets = load_dataset('/content/gdrive/MyDrive/dogImages/dogImages/train')
valid_files, valid_targets = load_dataset('/content/gdrive/MyDrive/dogImages/dogImages/valid')
test_files, test_targets = load_dataset('/content/gdrive/MyDrive/dogImages/dogImages/test')


#  The following codes resize the input to 224*224 pixels image and load it to memory as numpy series

In [6]:
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
from keras.utils import load_img, img_to_array

def path_to_tensor(img_path):
  #load RGB image as PIL.Image.Image type

  img=image.load_img(img_path,target_size=(224,224))

  #convert PIL.Image.Image type to 3D tensor with shape (224,224,3)

  x= image.img_to_array(img)
  #convert 3D tensor to 4D tensor with shape (1,224,224,3) and return 4D tensor

  return np.expand_dims(x,axis=0)

def paths_to_tensor(img_paths):
  list_of_tensors=[path_to_tensor(img_path) for img_path in tqdm(img_paths)]
  
  return np.vstack(list_of_tensors)

# Pre-process data
We need to normalize our data to eliminate units of measurement. Normalization can help our model better compare data of different scales.

In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

#Preprocess the data for Keras

train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:06<00:00, 123.23it/s]


#Create a model using transfer learning

In [8]:
import requests
url = 'https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz'
r = requests.get(url)
with open('bottleneck_features/DogResnet50Data.npz', 'wb') as f:
    f.write(r.content)
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')
train_Resnet50 = bottleneck_features['train']
valid_Resnet50 = bottleneck_features['valid']
test_Resnet50 = bottleneck_features['test']

FileNotFoundError: ignored